In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Conv2D, MaxPooling2D, Flatten
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tqdm import trange, notebook

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path ='/content/drive/MyDrive/K-DRIFT/AI_K-DRIFT/data_for_AI/'

In [ ]:
def load_data(base_path, num_samples=10000):
    wfm_list = []
    error_list = []

    for i in notebook.tqdm(range(1, num_samples + 1)):
        folder_path = os.path.join(base_path, f'data{i}')
        wfm_file = os.path.join(folder_path, 'wfm.txt')
        error_file = os.path.join(folder_path, 'error.txt')

        # WFM 데이터 로드 (17번째 줄부터 사용)
        with open(wfm_file, 'r',encoding = 'utf-16') as file:
            lines = file.readlines()
            wfm_data = np.array([list(map(float, line.split())) for line in lines[16:]])
            wfm_list.append(wfm_data)

        # Error 데이터 로드
        error_data = np.loadtxt(error_file)
        error_list.append(error_data)

    wfm_array = np.array(wfm_list)
    error_array = np.array(error_list)

    return wfm_array, error_array

In [ ]:
wfm_data, error_data = load_data(path)

  0%|          | 0/10000 [00:00<?, ?it/s]

In [ ]:
map_height = 64
map_width = 64
num_features = 6

In [ ]:
len(wfm_data)

In [ ]:
scaler = MinMaxScaler()
wavefront_maps_scaled = scaler.fit_transform(wfm_data.reshape(-1, map_height * map_width))
wavefront_maps_scaled = wavefront_maps_scaled.reshape(-1, map_height, map_width, 1)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(wavefront_maps_scaled, error_data, test_size=0.2, random_state=42)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# CNN 모델 정의
cnn_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(map_height, map_width, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(num_features)
])

cnn_model.compile(optimizer='adam', loss='mae', metrics=['mae'])

# 모델 학습
cnn_model.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.2)


Epoch 1/50
100/100 [==============================] - 39s 382ms/step - loss: 1.7455 - mae: 1.7455 - val_loss: 1.6500 - val_mae: 1.6500
Epoch 2/50
100/100 [==============================] - 34s 343ms/step - loss: 1.5805 - mae: 1.5805 - val_loss: 1.5918 - val_mae: 1.5918
Epoch 3/50
100/100 [==============================] - 36s 357ms/step - loss: 1.5226 - mae: 1.5226 - val_loss: 1.5166 - val_mae: 1.5166
Epoch 4/50
100/100 [==============================] - 34s 338ms/step - loss: 1.4498 - mae: 1.4498 - val_loss: 1.4298 - val_mae: 1.4298
Epoch 5/50
100/100 [==============================] - 34s 336ms/step - loss: 1.3941 - mae: 1.3941 - val_loss: 1.3513 - val_mae: 1.3513
Epoch 6/50
100/100 [==============================] - 33s 326ms/step - loss: 1.3193 - mae: 1.3193 - val_loss: 1.3507 - val_mae: 1.3507
Epoch 7/50
100/100 [==============================] - 33s 334ms/step - loss: 1.2670 - mae: 1.2670 - val_loss: 1.2695 - val_mae: 1.2695
Epoch 8/50
100/100 [==============================] - 3

In [ ]:
from tensorflow.keras.layers import LSTM

# RNN 모델 정의
rnn_model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(map_height, map_width)),
    LSTM(64),
    Dense(128, activation='relu'),
    Dense(num_features)
])

rnn_model.compile(optimizer='adam', loss='mae', metrics=['mae'])

# 모델 학습
rnn_model.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.2)


Epoch 1/50
10/10 [==============================] - 6s 193ms/step - loss: 2.4470 - mae: 2.4470 - val_loss: 2.3947 - val_mae: 2.3947
Epoch 2/50
10/10 [==============================] - 1s 87ms/step - loss: 2.2498 - mae: 2.2498 - val_loss: 2.1401 - val_mae: 2.1401
Epoch 3/50
10/10 [==============================] - 1s 98ms/step - loss: 2.0308 - mae: 2.0308 - val_loss: 1.9787 - val_mae: 1.9787
Epoch 4/50
10/10 [==============================] - 1s 98ms/step - loss: 1.8774 - mae: 1.8774 - val_loss: 1.8398 - val_mae: 1.8398
Epoch 5/50
10/10 [==============================] - 1s 97ms/step - loss: 1.7893 - mae: 1.7893 - val_loss: 1.7507 - val_mae: 1.7507
Epoch 6/50
10/10 [==============================] - 1s 91ms/step - loss: 1.7442 - mae: 1.7442 - val_loss: 1.7130 - val_mae: 1.7130
Epoch 7/50
10/10 [==============================] - 1s 116ms/step - loss: 1.6972 - mae: 1.6972 - val_loss: 1.6652 - val_mae: 1.6652
Epoch 8/50
10/10 [==============================] - 1s 143ms/step - loss: 1.6552 

In [ ]:
# 모델 평가
cnn_eval = cnn_model.evaluate(X_test, y_test)
rnn_eval = rnn_model.evaluate(X_test, y_test)

print(f'CNN Model MAE: {cnn_eval[1]}')
print(f'RNN Model MAE: {rnn_eval[1]}')

# 예측값과 실제값 비교
cnn_predictions = cnn_model.predict(X_test)
rnn_predictions = rnn_model.predict(X_test)

# 첫 5개 샘플의 예측값과 실제값 출력
for i in range(5):
    print(f'Sample {i+1}:')
    print(f'Actual Errors: {y_test[i]}')
    print(f'CNN Predicted Errors: {cnn_predictions[i]}')
    print(f'RNN Predicted Errors: {rnn_predictions[i]}')
    print()


7/7 [==============================] - 0s 21ms/step - loss: 1.6177 - mae: 1.6177
CNN Model MAE: 1.2658690214157104
RNN Model MAE: 1.6176652908325195
7/7 [==============================] - 1s 20ms/step
Sample 1:
Actual Errors: [ 1.907  0.756  2.63  -3.792  4.938 -2.169]
CNN Predicted Errors: [-2.472746    0.63242215  3.5195777  -3.6890392   2.3034618  -2.5062127 ]
RNN Predicted Errors: [-1.917227   0.1497048  3.0492332 -3.9649     2.5700145 -3.3934793]

Sample 2:
Actual Errors: [-1.811 -3.554 -3.132  2.612 -0.242  4.243]
CNN Predicted Errors: [ 0.42638943 -2.0286808  -4.210969    0.9975632  -1.9976084   2.7641108 ]
RNN Predicted Errors: [ 0.86569977 -2.8908942  -4.0689874   1.661438   -2.1828687   2.480794  ]

Sample 3:
Actual Errors: [ 3.443 -0.233  1.811 -3.466 -3.218 -3.511]
CNN Predicted Errors: [ 1.2933028   0.06833887  1.9125139  -3.4919064  -0.11347507  0.22210911]
RNN Predicted Errors: [ 0.90653217 -2.1755898   1.266045   -2.804521   -1.4962507   0.32004675]

Sample 4:
Actual Er